Модель для обработки CSV

In [13]:
import os
import pandas as pd
import glob
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Чтение и обработка данных валидационного датасета
def load_validation_data(validation_file_path):
    with open(validation_file_path, 'r') as f:
        lines = f.readlines()

    parsed_data = []

    for line in lines:
        cleaned_line = line.strip().replace('"', '').strip()
        split_line = cleaned_line.split(',')
        parsed_data.append(split_line)

    validation_df = pd.DataFrame(parsed_data, columns=['filename', 'position', 'is_header'])
    validation_df['is_header'] = pd.to_numeric(validation_df['is_header'], errors='coerce')
    validation_df['is_header'] = validation_df['is_header'].fillna(0).astype(int)

    return validation_df

# Функция для извлечения признаков для одной строки
def extract_single_row_features(row, row_index):
    num_cells = row.count()  # Число ячеек
    avg_cell_length = row.apply(lambda x: len(str(x))).mean()  # Средняя длина ячейки
    num_chars = row.apply(lambda x: len(str(x).replace(' ', ''))).sum()  # Число символов
    pct_numeric = row.apply(lambda x: sum(c.isdigit() for c in str(x))).sum() / num_chars if num_chars > 0 else 0
    pct_alpha = row.apply(lambda x: sum(c.isalpha() for c in str(x))).sum() / num_chars if num_chars > 0 else 0
    pct_symbol = row.apply(lambda x: sum(not c.isalnum() for c in str(x))).sum() / num_chars if num_chars > 0 else 0
    pct_numeric_cells = sum(row.apply(lambda x: str(x).replace('.', '').isdigit())) / num_cells
    pct_empty_cells = sum(row.isna()) / num_cells  # Процент пустых ячеек
    pct_uppercase_cells = sum(row.apply(lambda x: str(x).isupper())) / num_cells  # Процент ячеек с верхним регистром
    unique_char_count = row.apply(lambda x: len(set(str(x)))).sum()  # Количество уникальных символов
    return pd.Series([num_cells, avg_cell_length, num_chars, pct_numeric, pct_alpha, pct_symbol, pct_numeric_cells, pct_empty_cells, pct_uppercase_cells, unique_char_count, row_index])

# Функция для обработки одной таблицы с учетом меток заголовков
def process_table(file, validation_df):
    # Чтение данных из CSV, использование on_bad_lines для игнорирования ошибок
    df = pd.read_csv(file, sep=';', on_bad_lines='skip', engine='python')

    # Добавляем столбец 'is_header' и заполняем значениями 0
    df['is_header'] = 0

    # Получаем название файла для фильтрации меток заголовков
    filename = os.path.basename(file).replace('.csv', '')

    # Фильтруем строки из валидационного набора данных для текущего файла
    file_validation_data = validation_df[validation_df['filename'].str.contains(filename)]

    # Проставляем метки заголовков на основе информации из валидационного датасета
    for _, row in file_validation_data.iterrows():
        position = row['position'].strip()
        header_index = int(row['is_header'])

        if position == 'top':
            if header_index < len(df):
                df.loc[header_index, 'is_header'] = 1
        elif position == 'left':
            df.iloc[:, header_index] = 1

    # Извлечение признаков для каждой строки
    features_df = df.apply(lambda row: extract_single_row_features(row, df.index.get_loc(row.name)), axis=1)

    # Добавление признаков к оригинальной таблице
    feature_columns = ['num_cells', 'avg_cell_length', 'num_chars', 'pct_numeric', 'pct_alpha', 'pct_symbol', 'pct_numeric_cells', 'pct_empty_cells', 'pct_uppercase_cells', 'unique_char_count', 'row_index']
    features_df.columns = feature_columns

    # Объединяем признаки и метки заголовков
    df = pd.concat([features_df, df[['is_header']]], axis=1)

    return df

# Основная функция для обработки всех файлов
def process_all_files(data_folder, validation_file_path, output_path):
    # Загрузка данных валидационного набора
    validation_df = load_validation_data(validation_file_path)

    # Обработка всех CSV файлов
    files = glob.glob(os.path.join(data_folder, '*.csv'))
    all_data_with_features = []

    for file in files:
        processed_df = process_table(file, validation_df)  # Обработка каждой таблицы
        processed_df['filename'] = os.path.basename(file)  # Добавляем имя файла для идентификации
        all_data_with_features.append(processed_df)  # Добавляем таблицу в список

    # Конкатенация всех таблиц в один общий DataFrame
    fedot_dataset = pd.concat(all_data_with_features, ignore_index=True)

    # Сохранение итогового DataFrame с признаками в новый CSV файл
    fedot_dataset.to_csv(output_path, index=False, sep=';')
    print(f"Обработка завершена, файл сохранён как '{output_path}'")

# Пример использования:
data_folder = '/content/data'
validation_file_path = '/content/CTA_SAUS189_gt.csv'
output_path = 'fedot_training_dataset.csv'

process_all_files(data_folder, validation_file_path, output_path)

# Гиперпараметрический тюнинг для модели RandomForest
def model_tuning(X_train, y_train):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }

    rf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    print(f"Лучшие параметры: {grid_search.best_params_}")
    return grid_search.best_estimator_

# Пример вызова функции тюнинга (вставить после разделения данных)
# tuned_model = model_tuning(X_train, y_train)


Обработка завершена, файл сохранён как 'fedot_training_dataset.csv'


Попытка сбалансировать классы и использовать градиентный бустинг

In [9]:
pip install textdistance
pip install xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 5.3 MB/s eta 0:00:00


In [16]:
import os
import pandas as pd
import glob

# Функция для извлечения признаков для одной строки
def extract_single_row_features(row):
    num_cells = row.count()  # Число ячеек
    avg_cell_length = row.apply(lambda x: len(str(x))).mean()  # Средняя длина ячейки
    num_chars = row.apply(lambda x: len(str(x).replace(' ', ''))).sum()  # Число символов
    pct_numeric = row.apply(lambda x: sum(c.isdigit() for c in str(x))).sum() / num_chars if num_chars > 0 else 0
    pct_alpha = row.apply(lambda x: sum(c.isalpha() for c in str(x))).sum() / num_chars if num_chars > 0 else 0
    pct_symbol = row.apply(lambda x: sum(not c.isalnum() for c in str(x))).sum() / num_chars if num_chars > 0 else 0
    pct_numeric_cells = sum(row.apply(lambda x: str(x).replace('.', '').isdigit())) / num_cells
    pct_empty_cells = sum(row.isna()) / num_cells  # Процент пустых ячеек
    pct_uppercase_cells = sum(row.apply(lambda x: str(x).isupper())) / num_cells  # Процент ячеек с верхним регистром
    unique_char_count = row.apply(lambda x: len(set(str(x)))).sum()  # Количество уникальных символов

    return pd.Series([num_cells, avg_cell_length, num_chars, pct_numeric, pct_alpha, pct_symbol, pct_numeric_cells, pct_empty_cells, pct_uppercase_cells, unique_char_count])

# Функция для обработки одной таблицы (реальные данные)
def process_real_table(file):
    # Чтение данных из CSV
    df = pd.read_csv(file, sep=';', on_bad_lines='skip', engine='python')

    # Извлечение признаков для каждой строки
    features_df = df.apply(extract_single_row_features, axis=1)

    # Добавление признаков к оригинальной таблице
    feature_columns = ['num_cells', 'avg_cell_length', 'num_chars', 'pct_numeric', 'pct_alpha', 'pct_symbol', 'pct_numeric_cells', 'pct_empty_cells', 'pct_uppercase_cells', 'unique_char_count']
    features_df.columns = feature_columns

    # Возвращаем DataFrame с признаками
    return features_df

# Основная функция для обработки всех файлов реальных данных
def process_real_data(data_folder, output_path):
    # Обработка всех CSV файлов
    files = glob.glob(os.path.join(data_folder, '*.csv'))
    all_data_with_features = []

    for file in files:
        processed_df = process_real_table(file)  # Обработка каждой таблицы
        processed_df['filename'] = os.path.basename(file)  # Добавляем имя файла для идентификации
        all_data_with_features.append(processed_df)  # Добавляем таблицу в список

    # Конкатенация всех таблиц в один общий DataFrame
    real_data_with_features = pd.concat(all_data_with_features, ignore_index=True)

    # Сохранение итогового DataFrame с признаками в новый CSV файл
    real_data_with_features.to_csv(output_path, index=False, sep=';')
    print(f"Обработка завершена, файл сохранён как '{output_path}'")

# Пример использования:
data_folder = '/content/realdata'
output_path = 'real_data_with_features.csv'

process_real_data(data_folder, output_path)


Обработка завершена, файл сохранён как 'real_data_with_features.csv'


In [14]:
import pandas as pd
import numpy as np  # Добавляем numpy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
from sklearn.utils import class_weight

# Загрузка данных
df = pd.read_csv('fedot_training_dataset.csv', sep=';')

# Разделение признаков и целевой переменной
X = df.drop(columns=['is_header', 'filename'])
y = df['is_header']

# Разделение на обучающую и тестовую выборки (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Преобразуем список в numpy массив
class_weights = class_weight.compute_class_weight('balanced', classes=np.array([0, 1]), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# Модель Random Forest с балансировкой классов
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Отчет о классификации и точность для Random Forest
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf, zero_division=0))
print(f"Accuracy RF: {accuracy_score(y_test, y_pred_rf):.2f}")

# Модель SVM с балансировкой классов
svm_model = SVC(kernel='linear', class_weight='balanced', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Отчет о классификации и точность для SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm, zero_division=0))
print(f"Accuracy SVM: {accuracy_score(y_test, y_pred_svm):.2f}")

# Модель XGBoost с балансировкой классов
xgb_model = XGBClassifier(random_state=42, scale_pos_weight=class_weight_dict[0]/class_weight_dict[1])
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Отчет о классификации и точность для XGBoost
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print(f"Accuracy XGBoost: {accuracy_score(y_test, y_pred_xgb):.2f}")


Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1675
           1       0.84      0.88      0.86       187

    accuracy                           0.97      1862
   macro avg       0.91      0.93      0.92      1862
weighted avg       0.97      0.97      0.97      1862

Accuracy RF: 0.97
SVM Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      1675
           1       0.64      0.98      0.78       187

    accuracy                           0.94      1862
   macro avg       0.82      0.96      0.87      1862
weighted avg       0.96      0.94      0.95      1862

Accuracy SVM: 0.94
XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1675
           1       0.91      0.74      0.82       187

    accuracy                           0.97      1

In [17]:
import pandas as pd

# Загрузка реальных данных для предсказания
real_data_path = '/content/real_data_with_features.csv'
real_data = pd.read_csv(real_data_path, sep=';')

# Предположим, что реальные данные имеют те же признаки, что и тренировочные
X_real = real_data.drop(columns=['is_header', 'filename'], errors='ignore')

# Применение обученной модели (например, RandomForest)
y_pred_real_rf = rf_model.predict(X_real)

# Применение обученной модели (например, SVM)
y_pred_real_svm = svm_model.predict(X_real)

# Применение обученной модели (например, XGBoost)
y_pred_real_xgb = xgb_model.predict(X_real)

# Добавляем предсказания в исходный DataFrame
real_data['predicted_is_header_rf'] = y_pred_real_rf
real_data['predicted_is_header_svm'] = y_pred_real_svm
real_data['predicted_is_header_xgb'] = y_pred_real_xgb

# Сохраняем результаты в CSV
real_data.to_csv('/content/real_data_with_features.csv', index=False)

print("Прогнозы сохранены в файл 'predicted_real_data.csv'")


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- row_index
